<a href="https://colab.research.google.com/github/haeggee/error-detection-mt/blob/main/error_detection_in_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Critical Error Detection in Machine Translation with BERT, XLM-RoBERTa and HuggingFace*
### Introduction 

In this notebook, we tackle the problem of critical-error detection in machine translation. To that end, we show how to fine-tune different transformer based language models, including BERT and the multilingual XLM-RoBERTa model, and facilitate extensive evaluation. This PyTorch implementation leverages the Hugging face *transformers* to download pre-trained models, enable quick research experiments, access datasets and evaluation metrics.

This task is part of the WMT'21 [shared task on quality estimation](http://www.statmt.org/wmt21/quality-estimation-task.html).

---
### Task and Dataset
The goal of this task is to predict sentence-level binary scores indicating whether or not a translation contains (at least one) critical error. Translations with such errors are defined as translations that deviate in meaning as compared to the source sentence in such a way that they are misleading and may carry health, safety, legal, reputation, religious or financial implications. 

The data consists of Wikipedia comments in English extracted from two sources: the Jigsaw Toxic Comment Classification Challenge and the Wikipedia Comments Corpus, with translations generated by the ML50 multilingual translation model by FAIR. It contains instances in the following languages:

* English-Czech
* English-Japanese
* English-Chinese
* English-German

The dataset used in this notebook has been prepared by ourselves for the purpose of this task, and should be uploaded to the ```datasets``` directory as provided by the ```wmt21_multi_btr_{train,dev}.pkl``` in our [GitHub repository](https://github.com/haeggee/error-detection-mt).  

---

The main features of this notebook are: 
- End-to-end ML implementation (training, validation, prediction, evaluation)
- Easy adaptability
- Facilitation of quick experiments and extensions
- Quick training with limited computational resources (mixed-precision, gradient accumulation, ...)
- Threshold choice for the classification decision (not necessarily 0.5)
- Reproducible results with seed settings

Parts of this code have been taken and adapted from [NadirEM](https://github.com/NadirEM/nlp-notebooks/blob/master/Fine_tune_ALBERT_sentence_pair_classification.ipynb) and we thank the author for providing such a nice template. 

## Installation of libraries and imports

In [1]:
!pip install transformers==4.8.2 -q
!pip install sentencepiece -q
!pip install pickle5 -q

     |████████████████████████████████| 2.5 MB 15.7 MB/s 
     |████████████████████████████████| 895 kB 64.6 MB/s 
     |████████████████████████████████| 3.3 MB 62.5 MB/s 
     |████████████████████████████████| 1.2 MB 12.6 MB/s 
     |████████████████████████████████| 132 kB 12.0 MB/s 


In [3]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import pickle5 as pickle
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, \
                         get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, \
                            f1_score, precision_score, recall_score, \
                            confusion_matrix, matthews_corrcoef, \
                            precision_recall_curve
import gc # garbage collector
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
def printm():
  if torch.cuda.is_available():
    gpu = GPUs[0]
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + \
          humanize.naturalsize( psutil.virtual_memory().available ),\
          " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% \
          Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed,\
                                  gpu.memoryUtil*100, gpu.memoryTotal))
  else:
    print("No GPU in use.")
printm()

Gen RAM Free: 12.5 GB  | Proc size: 600.6 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0%           Total 16280MB


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In case GPU utilisation (Util) is not at 0%, you can uncomment and run the following line to kill all processes to get the full GPU afterwards. Make sure to comment out the line again to not constantly crash the notebook on purpose.

In [ ]:
#!kill -9 -1

## Loading the dataset

The easiest way to run the experiments on the dataset of WMT'21 is to clone our GitHub Repo [here](https://github.com/haeggee/mt-error-detection/) and inside the directory create a zip of the ```dataset/``` folder via

```
zip -q -r dataset dataset
```

Then upload the zip file to the instance of this Colab VM.

In [6]:
!unzip -qq -o dataset

In [7]:
# Load the MRPC dataset (train, validation and test)
filename_train = "dataset/wmt21_multi_btr_train.pkl" 
dataset_train = pickle.load(open(filename_train,'rb'))
filename_dev = "dataset/wmt21_multi_btr_dev.pkl" 
dataset_dev = pickle.load(open(filename_dev, 'rb'))

In [8]:
max_src = 0
avg_src = 0
max_mt = 0
avg_mt = 0
len_src = []
len_mt = []
len_btr = []
for src in dataset_train.src:
  len_src.append(len(src))
for mt in dataset_train.mt:
  len_mt.append(len(mt))
for btr in dataset_train.btr:
  len_btr.append(len(btr))
df_length = pd.DataFrame()
df_length["src"] = len_src
df_length["mt"] = len_mt
df_length["btr"] = len_btr
df_length.describe()

,src,mt,btr
count,29867.000000,29867.000000,29867.000000
mean,90.432986,78.861519,85.427227
std,28.269123,35.620897,30.758577
min,45.000000,12.000000,13.000000
25%,66.000000,52.000000,61.000000
50%,87.000000,72.000000,81.000000
75%,113.000000,102.000000,107.000000
max,148.000000,402.000000,984.000000


In [9]:
len(dataset_train)

29867

In [10]:
# split the original training data for validation
df_train, df_val = train_test_split(dataset_train,
                                    test_size = 0.05,
                                    random_state=42)
df_test = dataset_dev

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [11]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(28373, 8)
(1494, 8)
(3996, 8)


In [12]:
df_train.head()

,id,src,mt,list_scores,avg_scores,critical,language_pair,btr
0,2527,REDIRECT Talk:Royal Canadian Mounted Police Pr...,リダイレクト トーク : カナダ 王立 騎兵 警察 の 保護 警察,"[0, 0, 0]",NOT,0,en-ja,Redirect Talk : Protective Police of the Royal...
1,4131,He mentions the apartheid controversy in great...,Zmiňuje spor o apartheid velmi podrobně ve své...,"[0, 0, 0]",NOT,0,en-cs,He mentions the apartheid dispute in great det...
2,9447,"As you may have noticed, I am waiting until th...","Jak jste si možná všiml , čekám , až budou mén...","[0, 0, 0]",NOT,0,en-cs,"As you may have noticed, I'm waiting for them ..."
3,6168,Niteshift you are one seriously ignorant indiv...,ナイト シフト あなた は 1 つ の 深刻 な 無知 の 個人 笑 で す 。,"[0, 0, 0]",NOT,0,en-ja,Night Shift You are one serious ignorant indiv...
4,4888,RR down with relentless recussant!!! the empir...,RR nach unten mit unermüdlichen wiederkehrende...,"[0, 0, 0]",NOT,0,en-de,RR down with tireless recurring!!! the empire ...


## Classes and functions

In [13]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, bert_model='xlm-roberta-large',
                 with_labels=True, mask_prob=0.0, eval_set=False,
                 use_flip=False, mask_tokens=False):

        self.data = data  # pandas dataframe
        self.bert_model = bert_model
        # Initialize the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  
        self.maxlen = maxlen
        self.with_labels = with_labels 
        self.mask_prob = mask_prob
        self.eval_set = eval_set
        self.flip = use_flip
        self.mask_tokens = mask_tokens

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index
        # in the dataframe
        sent1 = str(self.data.loc[index, 'src'])
        if self.bert_model == 'bert-base-uncased':
          sent2 = str(self.data.loc[index, 'btr'])
        else:
          sent2 = str(self.data.loc[index, 'mt'])
        flip = (torch.rand(1) <= 0.5).int().item()
        if self.flip and (flip and not self.eval_set):
          sent1, sent2 = sent2, sent1
        # Tokenize the pair of sentences to get token ids, attention masks
        # and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length', # Pad to max_length
                                      truncation=True, # Truncate to max_length
                                      max_length=self.maxlen, 
                                      return_token_type_ids=True,
                                      return_tensors='pt') # Return tensors
        # tensor of token ids
        token_ids = encoded_pair['input_ids'].squeeze(0)
        
        # set fraction mask_prob of tokens to [MASK]
        if self.mask_tokens and (self.mask_prob > 0.0 and not self.eval_set):
          probability_matrix = torch.full(token_ids.shape, self.mask_prob)
          special_tokens_mask = \
                  self.tokenizer.get_special_tokens_mask(token_ids.tolist(), \
                  already_has_special_tokens=True)
          special_tokens_mask = \
                torch.tensor(special_tokens_mask, dtype=torch.bool)
          probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
          masked_indices = torch.bernoulli(probability_matrix).bool()
          token_ids[masked_indices] = \
                self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

  
        # binary tensor with "0" for padded values and "1" for the other values
        attn_masks = encoded_pair['attention_mask'].squeeze(0)
        
        # binary tensor with "0" for the 1st sentence tokens & "1"
        # for the 2nd sentence tokens. note xml-roberta doesn't make use it
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)
        
        if self.with_labels:  # True if the dataset has labels
            label = int(self.data.loc[index, 'critical'])
            return token_ids, attn_masks, token_type_ids, label, flip 
        else:
            return token_ids, attn_masks, token_type_ids, flip

In [14]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="xlm-roberta-base", freeze_bert=False,
                 num_classes=1):
        super(SentencePairClassifier, self).__init__()
        
        self.bert_model = bert_model
        # Instantiating BERT-based model object
        self.bert_layer = AutoModel.from_pretrained(bert_model)
        # Fix the hidden-state size of the encoder outputs 
        if bert_model == "xlm-roberta-base":
            hidden_size = 768
        elif bert_model == "xlm-roberta-large":
            hidden_size = 1024
        elif bert_model == "bert-base-uncased":
            hidden_size = 768
        elif bert_model.startswith('TransQuest'):
            hidden_size = 1024
        else:
            raise Exception('Unsupported model for this notebook.')

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer for error
        self.cls_layer = nn.Linear(hidden_size, num_classes)
        # Classification layer for flipping
        self.cls_layer_flip = nn.Linear(hidden_size, num_classes)

        self.dropout = nn.Dropout(p=0.5)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids=None):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to
                          focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to
                              identify sentence1 and sentence2 (only used by 
                              BERT, not XLM)
        '''

        bert_output = self.bert_layer(input_ids, attn_masks)
        pooler_output = bert_output.pooler_output
        
        # Feeding to the classifier layer the last layer hidden-state of the
        # [CLS] token further processed by a Linear Layer and a Tanh activation.
        logits = self.cls_layer(self.dropout(pooler_output))
        logits_flip = self.cls_layer_flip(self.dropout(pooler_output))
        return logits, logits_flip

In [15]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by
    applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    """ 
    Evaluate the loss for the critial error classification wrt. 
    a specific criterion and dataloader, averaged over all samples.
    Does not collect gradients.
    """
    net.eval()

    mean_loss = 0
    count = 0
    probs_all = []
    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels, labels_flip) in \
                enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), \
                token_type_ids.to(device), labels.to(device)
              
            logits, _ = net(seq, attn_masks, token_type_ids)
            probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
            probs_all += probs.tolist()
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count, np.array(probs_all)

In [16]:
print("Creation of the models' folder...")
!mkdir -p models/TransQuest

Creation of the models' folder...


Link for mixed precision training, gradient scaling and gradient accumulation  : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples

If you would like to learn more about Training Neural Nets on Larger Batches, I suggest reading this post of Thomas Wolf :
https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255

In [17]:
def train_bert(bert_model, net, criterions, opti, lr, lr_scheduler, train_loader,
               minitrain_loader, labels_minitrain, val_loader, labels_val, 
               epochs, iters_to_accumulate, use_flip):
    """
    Train the model for a specified set of parameters.
    In particular, `net` is a SentencePairClassifier as specified above,
    and criterions is a tuple (crit_1, crit_2) where crit_1 is the 
    criterion of interest (here, BCE for error classification).
    
    Moreover, we evaluate the model after every epoch for the validation set
    as well as a small fraction of the training set and print selected metrics.
    """
    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    # print the training loss two times per epoch
    print_every = nb_iterations // 2
    iters = []
    train_losses = []
    val_losses = []
    
    # the two different tasks we train on
    criterion = criterions[0]
    criterion_flip = criterions[1]
    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        probs_all = []
        for it, (seq, attn_masks, token_type_ids, labels, labels_flip) in \
            enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels, labels_flip = \
                seq.to(device), attn_masks.to(device), \
                token_type_ids.to(device), labels.to(device),\
                labels_flip.to(device)
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits, logits_flip = net(seq, attn_masks, token_type_ids)
                # Computing loss
                loss = criterion(logits.squeeze(-1),
                                    labels.float())
                if use_flip:
                  loss_flip = criterion_flip(logits_flip.squeeze(-1),
                                         labels_flip.float())
                  loss = (loss + loss_flip)
                # Normalize the loss because it is averaged
                loss = loss / iters_to_accumulate 
            # Backpropagating the gradients
            # Scales loss. Calls backward() on scaled loss
            # to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # --- Optimization step
                # scaler.step() first unscales the gradients of the optimizer's
                # assigned params. If these gradients do not contain infs or
                # NaNs, opti.step() is then called. Otherwise, opti.step()
                # is skipped.
                # Importantly, we have to check if the optimizer step was
                # skipped in order to also skip the lr_sched update.
                scaler.step(opti)
                scale = scaler.get_scale()
                # Updates the scale for next iteration.
                scaler.update()
                # if scale was changed means we skipped opt.step()
                skip_lr_schedule = (scale != scaler.get_scale())
                # Adjust the learning rate based on the number of iterations.
                if not skip_lr_schedule:
                  lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item() * iters_to_accumulate

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations,
                              ep+1, running_loss / print_every))

                running_loss = 0.0

        # Compute validation loss
        val_loss, probs_val = evaluate_loss(net, device, criterion, val_loader)
        preds_val = probs_val >= 0.5
        print()
        print("Epoch {} complete!\nValidation Loss : {}".format(ep+1, val_loss))
        print("Validation F1 score: {:.4f}, Recall: {:.4f}, MCC: {:.4f}".format(
                f1_score(labels_val, preds_val),
                recall_score(labels_val, preds_val),
                matthews_corrcoef(labels_val, preds_val)))
        
        # Compute loss on small fraction of training set
        minitrain_loss, probs_minitrain = evaluate_loss(net,
                                                        device,
                                                        criterion,
                                                        minitrain_loader)
        preds_minitrain = probs_minitrain >= 0.5
        print()
        print("Minitrain F1 score: {:.4f}, Recall: {:.4f}, MCC: {:.4f}".format(
                f1_score(labels_minitrain, preds_minitrain),
                recall_score(labels_minitrain, preds_minitrain),
                matthews_corrcoef(labels_minitrain, preds_minitrain)))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}"
                  .format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

        torch.cuda.empty_cache()
    
    # Saving the model
    path_to_model = 'models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(
        bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()
    return path_to_model

## Parameters

In [18]:
# bert_model = "xlm-roberta-base"
bert_model = "TransQuest/monotransquest-da-multilingual"
# freeze the encoder weights and only update the classification layer weights
freeze_bert = False
maxlen = 200  # 75% below
bs = 4 if bert_model.endswith('large') or bert_model.startswith('TransQuest') \
       else 16  # batch size

# the gradient accumulation adds gradients over an
# effective batch of size: bs * iters_to_accumulate.
# If set to "1", you get the usual batch size
iters_to_accumulate = 10
lr = 2e-5 # learning rate
epochs = 5  # number of training epochs
size_minitrain = 2000
weight_decay = 1e-2
mask_prob = 0.2 # percentage of tokens in sentence that we mask

# increase weight for pos label for data imbalance
pos_weight = ((df_train['critical'] == 0).sum() /
              (df_train['critical'] == 1).sum())
pos_weight = torch.Tensor([pos_weight.item()])
print('Positive weight for imbalance:', pos_weight)

Positive weight for imbalance: tensor([4.6262])


## Training and validation

Link for the AdamW optimizer and the learning rate scheduler :
https://huggingface.co/transformers/main_classes/optimizer_schedules.html

Below we define a function that does the full training depending on the which pretrained model to use as well as to the methods of random sentence flipping as well as masking of tokens. The other hyperparameters are defined above. 

In [ ]:
def train(bert_model, use_flip=False, mask_tokens=False, seed=1):
  #  Set all seeds to make reproducible results
  set_seed(seed)
  # Creating instances of training and validation set
  print("Reading training data...")
  train_set = CustomDataset(df_train, maxlen, bert_model,
                            use_flip=use_flip, mask_tokens=mask_tokens,
                            mask_prob=mask_prob)
  minitrain_set = CustomDataset(df_train[:size_minitrain], maxlen,
                                bert_model, eval_set=True)
  # Creating instances of the dataloaders
  train_loader = DataLoader(train_set, batch_size=bs, num_workers=2)
  minitrain_loader = DataLoader(minitrain_set, batch_size=bs, num_workers=2)

  print("Reading validation data...")
  val_set = CustomDataset(df_val, maxlen, bert_model, eval_set=True)
  val_loader = DataLoader(val_set, batch_size=bs, num_workers=2)

  labels_val = df_val['critical']  # true labels
  labels_minitrain = df_train[:size_minitrain]['critical']  # true labels
  net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)
  print("Total number of parameters:",
        sum(p.numel() for p in net.bert_layer.parameters()))

  net.to(device)

  criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))
  criterion_flip = nn.BCEWithLogitsLoss()
  opti = AdamW(net.parameters(), lr=lr, weight_decay=weight_decay)
  # The total number of training steps
  num_training_steps = epochs * len(train_loader)
  # The number of steps for the warmup phase.
  # num_warmup_steps = 0
  num_warmup_steps = int(0.1 * num_training_steps)
  # Necessary to take into account Gradient accumulation
  t_total = (len(train_loader) // iters_to_accumulate) * epochs
  lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti,
                num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

  path_to_model = train_bert(bert_model, net, [criterion, criterion_flip], opti,
                            lr, lr_scheduler, train_loader, minitrain_loader,
                            labels_minitrain, val_loader, labels_val,
                            epochs, iters_to_accumulate, use_flip)
  return net, path_to_model

An example call to training, with parameters as defined above:

In [ ]:
net, path_to_model = train(bert_model, use_flip=True, mask_tokens=True)

Reading training data...
Reading validation data...


Some weights of the model checkpoint at TransQuest/monotransquest-da-multilingual were not used when initializing XLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of parameters: 559890432


 20%|█▉        | 1418/7094 [07:26<28:54,  3.27it/s]


Iteration 1418/7094 of epoch 1 complete. Loss : 1.8825710001066642 


...


Iteration 7090/7094 of epoch 5 complete. Loss : 0.8807951407848591 


  0%|          | 0/500 [00:00<?, ?it/s]


Epoch 5 complete!
Validation Loss : 1.2350023027887955
Validation F1 score: 0.4656, Recall: 0.4388, MCC: 0.3535


100%|██████████| 500/500 [00:56<00:00,  8.93it/s]



Minitrain F1 score: 0.6174, Recall: 0.6058, MCC: 0.5397
The model has been saved in models/TransQuest/monotransquest-da-multilingual_lr_2e-05_val_loss_1.01851_ep_3.pt


You can download the model saved in the folder "models" by browsing the files on the left of the colab notebook.

Please note that you might run out of CUDA memory. As we cannot control the GPU assignments on Colab, it is possible to have GPUs that do not have enough RAM to handle the size of the pretrained models. In our experiments, at least 15-16 GB was used.

In [ ]:
# If you encounter a CUDA out of memory error: 
# - uncomment the kill command, run the "kill" command (and comment it)
# - reduce the batch size or maxlen parameters
# - then run all cells from the begining 

# If you get an ugly print of tqdm (all iterations are showed), follow the
# above first and last steps
printm()
# !kill -9 -1

Gen RAM Free: 11.1 GB  | Proc size: 6.3 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0%           Total 16280MB


## Prediction

In [19]:
print("Creation of the results' folder...")
!mkdir -p results

Creation of the results' folder...


In [20]:
def test_prediction(net, device, dataloader,
                    with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with
    or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, tt_ids, _, _ in tqdm(dataloader):
                seq, attn_masks, tt_ids = seq.to(device), \
                     attn_masks.to(device), tt_ids.to(device)

                logits, _ = net(seq, attn_masks, tt_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, tt_ids, _ in tqdm(dataloader):
                seq, attn_masks, tt_ids = seq.to(device), \
                     attn_masks.to(device), tt_ids.to(device)

                logits, _ = net(seq, attn_masks, tt_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [21]:
def load_and_predict(bert_model, test_loader, path_to_model, path_to_output):
  """
  Create a model and load weights from previous training, then
  predict the probabilities for a test_loader.
  """
  model = SentencePairClassifier(bert_model)
  if torch.cuda.device_count() > 1:  # if multiple GPUs
      print("Let's use", torch.cuda.device_count(), "GPUs!")
      model = nn.DataParallel(model)

  print()
  print("Loading the weights of the model...")
  model.load_state_dict(torch.load(path_to_model))

  model.to(device)

  print("Predicting on test data...")
  test_prediction(net=model, device=device, dataloader=test_loader,
                  with_labels=True, 
                  result_file=path_to_output)
  print()
  print("Predictions are available in : {}".format(path_to_output))

In [22]:
def evaluate_pred_test(labels_test, preds_test):
  accuracy = accuracy_score(labels_test, preds_test)
  bac = balanced_accuracy_score(labels_test, preds_test, adjusted=True)
  f1 = f1_score(labels_test, preds_test)
  precision = precision_score(labels_test, preds_test)
  recall = recall_score(labels_test, preds_test)
  cnf = confusion_matrix(labels_test, preds_test)
  mcc = matthews_corrcoef(labels_test, preds_test)
  print("-----Evaluation-----")
  if cnf.shape == (2,2):
    print("TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}".
          format(tp=cnf[1][1], tn=cnf[0][0], fp=cnf[0][1], fn=cnf[1][0]))
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
  print("MCC: ", mcc)
  print("Accuracy: ", accuracy)

def best_threshold(labels, probs, use_f1=True):
  if use_f1:
    precision, recall, thresholds = precision_recall_curve(labels, probs)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.nanargmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    threshold = thresholds[ix]
  else:
    thresholds = np.linspace(0, 1, 20)
    mcc_thres = np.array([matthews_corrcoef(labels,
                        (probs >= t).astype('uint8')) for t in thresholds])
    ix = np.nanargmax(mcc_thres)
    print('Best Threshold=%f, MCC=%.3f' % (thresholds[ix], mcc_thres[ix]))
    threshold = thresholds[ix]
  return threshold

We share below our pretrained models in order to reproduce results.

We use the model parameters that resulted in the best validation loss (saved to ```path_to_model```). Here, one can also use other pretrained models to directly compare.

You can download them and upload the ```model.pt``` files in the *models* folder and edit the ```path_to_model``` variable. Link:

** TODO: add drive link here **

---
The results included in the output of the cells below belong to the model as defined above. That is, we use
* TransQuest pretrained weights from Direct Assessment (DA) training in a transfer-learning fashion
* random flipping of the training sentences
* random masking of 20\% of tokens


In [23]:
# Choose threshold to maximize precision_recall (F1), else to maximize MCC
use_f1 = True

In [ ]:
predict_on_train_set = False
if predict_on_train_set:
  print("Reading training data without masking...")
  train_set = CustomDataset(df_train, maxlen, bert_model, mask_prob=mask_prob,
                            eval_set=True)
  train_loader = DataLoader(train_set, batch_size=bs, num_workers=2)

  print("Predicting on train data...")
  test_prediction(net=net, device=device, dataloader=train_loader,
                  with_labels=True, result_file='results/output_train.txt')
  print()
  labels_train = df_train['critical']
  probs_train = pd.read_csv('results/output_train.txt', header=None)[0]
  threshold = best_threshold(labels_train, probs_train, use_f1)
  preds_train = (probs_train >= threshold).astype('uint8')
  print("Eval on Training data:")
  evaluate_pred_test(labels_train, preds_train)

In [26]:
# path_to_model = .... # other models here
path_to_output_file = 'results/output.txt'
print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model, eval_set=True)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=2)
load_and_predict(bert_model, test_loader, path_to_model, path_to_output_file)

Reading test data...


Some weights of the model checkpoint at TransQuest/monotransquest-da-multilingual were not used when initializing XLMRobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Loading the weights of the model...


  0%|          | 0/999 [00:00<?, ?it/s]

Predicting on test data...


100%|██████████| 999/999 [01:49<00:00,  9.15it/s]


Predictions are available in : results/output.txt


You can download the predictions saved in the folder "results" by browsing the files on the left of the Colab Notebook.

## Evaluation

In [ ]:
def eval_lang_pairs(preds_test, df_test):
  language_pairs = ['en-cs', 'en-ja', 'en-zh', 'en-de']
  for lang in language_pairs:
    df_test_lang = df_test[df_test['language_pair'] == lang]
    if len(df_test_lang)==0:
      continue
    labels_test_lang = df_test_lang['critical'] 
    preds_test_lang = preds_test[df_test_lang.index.tolist()]
    print("\n For language pair {}".format(lang))
    evaluate_pred_test(labels_test_lang, preds_test_lang)

You can also directly use the predicted probabilities saved to a file such as we have saved above. The next cell loads the predictions and computes the best threshold in order to maximize the F1 score.

Our other predictions can be found here:

** TODO: add link here **

In [ ]:
labels_test = df_test['critical']  # true labels
#### evaluation
probs_test = pd.read_csv(path_to_output_file, header=None)[0]
# best threshold
threshold = best_threshold(labels_test, probs_test, use_f1)
preds_test = (probs_test>=threshold).astype('uint8')

Best Threshold=0.420166, F-Score=0.494


Link for the threshold choice problem : https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

In [ ]:
evaluate_pred_test(labels_test, preds_test)

-----Evaluation-----
TP: 401, TN: 2772, FP: 546, FN: 277
F1:  0.49353846153846154
Precision:  0.42344244984160506
Recall:  0.5914454277286135
MCC:  0.37680248358097124
Accuracy:  0.794044044044044


Evaluation for each language pair

In [ ]:
eval_lang_pairs(preds_test, df_test)


 For language pair en-cs
-----Evaluation-----
TP: 88, TN: 737, FP: 102, FN: 72
F1:  0.5028571428571429
Precision:  0.4631578947368421
Recall:  0.55
MCC:  0.4003745260589834
Accuracy:  0.8258258258258259

 For language pair en-ja
-----Evaluation-----
TP: 32, TN: 814, FP: 89, FN: 64
F1:  0.2949308755760369
Precision:  0.2644628099173554
Recall:  0.3333333333333333
MCC:  0.2120741129705655
Accuracy:  0.8468468468468469

 For language pair en-zh
-----Evaluation-----
TP: 85, TN: 686, FP: 172, FN: 56
F1:  0.42713567839195976
Precision:  0.33073929961089493
Recall:  0.6028368794326241
MCC:  0.3204874383384705
Accuracy:  0.7717717717717718

 For language pair en-de
-----Evaluation-----
TP: 196, TN: 535, FP: 183, FN: 85
F1:  0.5939393939393939
Precision:  0.5171503957783641
Recall:  0.697508896797153
MCC:  0.4101521033038911
Accuracy:  0.7317317317317318


## Comparison of different methods


In [ ]:
freeze_bert = False
maxlen = 200  # 75% below
bs = 4 # batch size

# the gradient accumulation adds gradients over an
# effective batch of size: bs * iters_to_accumulate.
# If set to "1", you get the usual batch size
iters_to_accumulate = 10
lr = 2e-5 # learning rate
epochs = 5  # number of training epochs
size_minitrain = 2000
weight_decay = 1e-2
mask_prob = 0.2 # percentage of tokens in sentence that we mask

# increase weight for pos label for data imbalance
pos_weight = ((df_train['critical'] == 0).sum() /
              (df_train['critical'] == 1).sum())
pos_weight = torch.Tensor([pos_weight.item()])
print('Positive weight for imbalance:', pos_weight)

Positive weight for imbalance: tensor([4.6262])


In [ ]:
# to free up as much memory as possible for the GPU,
# garbage collect + empty cache
gc.collect()
torch.cuda.empty_cache()
# tqdm._instances.clear()

In [ ]:
use_f1 = True

# Configs for experiments: (model, use_flip, mask_tokens)
models = [
          ('bert-base-uncased', False, False),
          ('xlm-roberta-large', False, False),
          ('TransQuest/monotransquest-da-multilingual', False, False),
          ('TransQuest/monotransquest-da-multilingual', True, False),
          ('TransQuest/monotransquest-da-multilingual', False, True),
          # this was already done above:
          #('TransQuest/monotransquest-da-multilingual', True, True),
          
]

In [ ]:
def full_experiment(model_name, use_flip, mask_tokens): 
  #### training
  print("--------------- Using Model:", model_name)
  print("--------------- Flipping sentences:", use_flip,
        ", Masking tokens:", mask_tokens)
  _, path_to_model = train(model_name,
                          use_flip=use_flip, mask_tokens=mask_tokens)
  path_to_output = "predictions_"
  path_to_output += model_name + ("-flip" if use_flip else "-noflip") + \
                                 ("-masked" if mask_tokens else "-unmasked") + \
                                 ".txt"
  path_to_output = path_to_output.replace("/", "-")
  #### testing
  test_set = CustomDataset(df_test, maxlen, model_name, eval_set=True)
  test_loader = DataLoader(test_set, batch_size=bs, num_workers=2)
  load_and_predict(model_name, test_loader, path_to_model, path_to_output)
  labels_test = df_test['critical']  # true labels
  #### evaluation
  probs_test = pd.read_csv(path_to_output, header=None)[0]
  # best threshold
  threshold = best_threshold(labels_test, probs_test, use_f1)
  preds_test = (probs_test>=threshold).astype('uint8')
  print("Predictions for ", model_name, "flip:", use_flip, ", masking:", mask_tokens)
  evaluate_pred_test(labels_test, preds_test)
  eval_lang_pairs(preds_test, df_test)
  

Please not that in case you get memory errors with CUDA, the best procedure is to restart the runtime, rerun the definitions and repeat only the one experiment you want to execute.

In [ ]:
## BERT with backtranslations
bert_exp = models[0]
full_experiment(bert[0], bert[1], bert[2])
gc.collect()
torch.cuda.empty_cache()
tqdm._instances.clear()

# XLM-RoBERTa
xlm = models[1]
full_experiment(xlm[0], xlm[1], xlm[2])
gc.collect()
torch.cuda.empty_cache()
tqdm._instances.clear()

--------------- Using Model: bert-base-uncased
--------------- Flipping sentences: False , Masking tokens: False
Reading training data...



Reading validation data...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of parameters: 109482240


 50%|█████     | 3550/7094 [02:28<02:28, 23.86it/s]

...


Loading the weights of the model...


  0%|          | 0/999 [00:00<?, ?it/s]

Predicting on test data...


100%|██████████| 999/999 [00:34<00:00, 29.27it/s]



Predictions are available in : predictions_xlm-roberta-large-noflip-unmasked.txt
Best Threshold=0.438721, F-Score=0.469
Predictions for  xlm-roberta-large flip: False , masking: False
-----Evaluation-----
TP: 360, TN: 2820, FP: 498, FN: 318
F1:  0.46875000000000006
Precision:  0.4195804195804196
Recall:  0.5309734513274337
MCC:  0.34815653689006015
Accuracy:  0.7957957957957958

 For language pair en-cs
-----Evaluation-----
TP: 82, TN: 741, FP: 98, FN: 78
F1:  0.4823529411764706
Precision:  0.45555555555555555
Recall:  0.5125
MCC:  0.37759178703547336
Accuracy:  0.8238238238238238

 For language pair en-ja
-----Evaluation-----
TP: 16, TN: 876, FP: 27, FN: 80
F1:  0.2302158273381295
Precision:  0.37209302325581395
Recall:  0.16666666666666666
MCC:  0.1986074452746329
Accuracy:  0.8928928928928929

 For language pair en-zh
-----Evaluation-----
TP: 50, TN: 757, FP: 101, FN: 91
F1:  0.34246575342465757
Precision:  0.33112582781456956
Recall:  0.3546099290780142
MCC:  0.23026155459142109
A

In [ ]:
## XLM-RoBERTa with TransQuest transfer-learning
tq = models[2]
full_experiment(tq[0], tq[1], tq[2])
gc.collect()
torch.cuda.empty_cache()
tqdm._instances.clear()

--------------- Using Model: TransQuest/monotransquest-da-multilingual
--------------- Flipping sentences: False , Masking tokens: False
Reading training data...
Reading validation data...


Some weights of the model checkpoint at TransQuest/monotransquest-da-multilingual were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of parameters: 559890432


 50%|█████     | 3548/7094 [06:54<06:34,  9.00it/s]


Iteration 3547/7094 of epoch 1 complete. Loss : 1.1563235866538666 


...


Loading the weights of the model...


  0%|          | 0/999 [00:00<?, ?it/s]

Predicting on test data...


100%|██████████| 999/999 [00:32<00:00, 30.54it/s]



Predictions are available in : predictions_TransQuest-monotransquest-da-multilingual-noflip-unmasked.txt
Best Threshold=0.458252, F-Score=0.487
Predictions for  TransQuest/monotransquest-da-multilingual flip: False , masking: False
-----Evaluation-----
TP: 370, TN: 2845, FP: 473, FN: 308
F1:  0.48652202498356345
Precision:  0.4389086595492289
Recall:  0.5457227138643068
MCC:  0.37090458010328053
Accuracy:  0.8045545545545546

 For language pair en-cs
-----Evaluation-----
TP: 84, TN: 756, FP: 83, FN: 76
F1:  0.5137614678899083
Precision:  0.5029940119760479
Recall:  0.525
MCC:  0.4187981813100839
Accuracy:  0.8408408408408409

 For language pair en-ja
-----Evaluation-----
TP: 28, TN: 834, FP: 69, FN: 68
F1:  0.2901554404145078
Precision:  0.28865979381443296
Recall:  0.2916666666666667
MCC:  0.21426094292254913
Accuracy:  0.8628628628628628

 For language pair en-zh
-----Evaluation-----
TP: 58, TN: 735, FP: 123, FN: 83
F1:  0.36024844720496896
Precision:  0.32044198895027626
Recall:  0

In [ ]:
# XLM-RoBERTa with TransQuest transfer-learning + flipping
tq_flip = models[3]
full_experiment(tq_flip[0], tq_flip[1], tq_flip[2])
gc.collect()
torch.cuda.empty_cache()
tqdm._instances.clear()

--------------- Using Model: TransQuest/monotransquest-da-multilingual
--------------- Flipping sentences: True , Masking tokens: False
Reading training data...
Reading validation data...


Some weights of the model checkpoint at TransQuest/monotransquest-da-multilingual were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of parameters: 559890432


 50%|█████     | 3548/7094 [05:52<05:39, 10.44it/s]


Iteration 3547/7094 of epoch 1 complete. Loss : 1.8602215504777375 


...


Loading the weights of the model...


  0%|          | 0/999 [00:00<?, ?it/s]

Predicting on test data...


100%|██████████| 999/999 [00:33<00:00, 30.03it/s]



Predictions are available in : predictions_TransQuest-monotransquest-da-multilingual-flip-unmasked.txt
Best Threshold=0.415771, F-Score=0.503
Predictions for  TransQuest/monotransquest-da-multilingual flip: True , masking: False
-----Evaluation-----
TP: 399, TN: 2809, FP: 509, FN: 279
F1:  0.5031525851197982
Precision:  0.43942731277533037
Recall:  0.588495575221239
MCC:  0.389718012185608
Accuracy:  0.8028028028028028

 For language pair en-cs
-----Evaluation-----
TP: 90, TN: 752, FP: 87, FN: 70
F1:  0.5341246290801187
Precision:  0.5084745762711864
Recall:  0.5625
MCC:  0.44070369358817907
Accuracy:  0.8428428428428428

 For language pair en-ja
-----Evaluation-----
TP: 24, TN: 856, FP: 47, FN: 72
F1:  0.2874251497005988
Precision:  0.3380281690140845
Recall:  0.25
MCC:  0.22705686090165325
Accuracy:  0.8808808808808809

 For language pair en-zh
-----Evaluation-----
TP: 69, TN: 729, FP: 129, FN: 72
F1:  0.4070796460176991
Precision:  0.3484848484848485
Recall:  0.48936170212765956
MC

In [ ]:
# XLM-RoBERTa with TransQuest transfer-learning + no flipping + masking
tq_mask = models[3]
full_experiment(tq_mask[0], tq_mask[1], tq_mask[2])
gc.collect()
torch.cuda.empty_cache()
tqdm._instances.clear()

--------------- Using Model: TransQuest/monotransquest-da-multilingual
--------------- Flipping sentences: False , Masking tokens: True
Reading training data...
Reading validation data...


Some weights of the model checkpoint at TransQuest/monotransquest-da-multilingual were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of parameters: 559890432


 50%|█████     | 3548/7094 [05:54<05:44, 10.28it/s]


Iteration 3547/7094 of epoch 1 complete. Loss : 1.1586007820234892 


...


Loading the weights of the model...


  0%|          | 0/999 [00:00<?, ?it/s]

Predicting on test data...


100%|██████████| 999/999 [00:37<00:00, 26.85it/s]



Predictions are available in : predictions_TransQuest-monotransquest-da-multilingual-noflip-masked.txt
Best Threshold=0.474609, F-Score=0.496
Predictions for  TransQuest/monotransquest-da-multilingual flip: False , masking: True
-----Evaluation-----
TP: 378, TN: 2849, FP: 469, FN: 300
F1:  0.49573770491803276
Precision:  0.4462809917355372
Recall:  0.5575221238938053
MCC:  0.38220623632355705
Accuracy:  0.8075575575575575

 For language pair en-cs
-----Evaluation-----
TP: 89, TN: 746, FP: 93, FN: 71
F1:  0.52046783625731
Precision:  0.489010989010989
Recall:  0.55625
MCC:  0.4232024010570228
Accuracy:  0.8358358358358359

 For language pair en-ja
-----Evaluation-----
TP: 33, TN: 840, FP: 63, FN: 63
F1:  0.34375
Precision:  0.34375
Recall:  0.34375
MCC:  0.27398255813953487
Accuracy:  0.8738738738738738

 For language pair en-zh
-----Evaluation-----
TP: 68, TN: 704, FP: 154, FN: 73
F1:  0.37465564738292007
Precision:  0.3063063063063063
Recall:  0.48226950354609927
MCC:  0.253569329223